# Cluster Analysis - Grouping Similar incorrect responses#

## Import dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Specify the path to the CSV file on Google Drive
dataset = "/content/drive/MyDrive/update_data_clean.csv"

# Read the CSV file into a Pandas DataFrame
df1 = pd.read_csv(dataset) # 'df' contains the data from the CSV file on Google Drive

In [ ]:
df1.head()

,TestNo,Username,Started.on,Completed,Time.taken,Grade.6.00,Q1_seed,Q1_problem,Q1_solution,Q1_mistakes,...,Q6_ans,Q6_validity,Q6_score,Q6_prt,Q1_anticipate,Q2_anticipate,Q3_anticipate,Q4_anticipate,Q5_anticipate,Q6_anticipate
0,T0,user004,10-10-22 10:10,10-10-22 10:57,46 mins 30 secs,5.0,808626,\(\int{\frac{4}{p}} d{a}\),4*log(abs(p))+c,\({4\cdot \ln \left( \left| p\right| \right)}\),...,7/2*(x^2 - 3)^(6/7) + c,score,1.0,# = 1 | ATInt_true. | prt1-1-T,0,-1,-1,-1,-1,-1
1,T1,user004,10-10-22 14:16,10-10-22 14:35,18 mins 34 secs,3.0,490862,\(\int{\frac{10}{y}} d{a}\),10*log(abs(y))+c,\({10\cdot \ln \left( \left| y\right| \right)}\),...,(x^2 -5)^(2/3)/(2/3) +c,score,1.0,# = 1 | ATInt_true. | prt1-1-T,-1,-1,-1,0,0,-1
2,T2,user006,10-08-22 14:07,10-09-22 0:53,10 hours 45 mins,1.0,397885,\(\int{\frac{9}{r}} d{a}\),9*log(abs(r))+c,\({9\cdot \ln \left( \left| r\right| \right)}\),...,(9/2)*(x^2-4)/((2/3)*(x^2-4)^(3/2))+c,score,0.0,# = 0 | ATInt_generic. | prt1-1-F | prt1-2-F,0,0,-1,0,0,0
3,T3,user006,10-09-22 8:37,10-09-22 9:10,32 mins 59 secs,3.0,154367,\(\int{\frac{13}{t}} d{a}\),13*log(abs(t))+c,\({13\cdot \ln \left( \left| t\right| \right)}\),...,7/2*u^(6/7)+c,score,0.0,# = 0 | ATInt_generic. ATInt_var_SB_notSA. | p...,0,-1,-1,0,-1,0
4,T4,user006,10-09-22 9:15,10-09-22 9:43,28 mins 18 secs,5.0,342032,\(\int{\frac{15}{t}} d{a}\),15*log(abs(t))+c,\({15\cdot \ln \left( \left| t\right| \right)}\),...,7/6*(x^2-4)^(6/7)+c,score,1.0,# = 1 | ATInt_true. | prt1-1-T,0,-1,-1,0,-1,-1


In [ ]:
df1.set_index('TestNo', inplace=True)

In [ ]:
anticipated_counts = {}

# Loop through each question type
for i in range(1, 7):
    anticipate_col_name = f'Q{i}_anticipate'

    # Filter rows where the column value is 1 and count them
    anticipated_counts[f'Q{i}_solution'] = df1[df1[anticipate_col_name] == 1].shape[0]

print(anticipated_counts)


{'Q1_solution': 32, 'Q2_solution': 4, 'Q3_solution': 221, 'Q4_solution': 77, 'Q5_solution': 1, 'Q6_solution': 53}


In [ ]:
not_anticipated_counts = {}

# Loop through each question type
for i in range(1, 7):
    anticipate_col_name = f'Q{i}_anticipate'

    # Filter rows where the column value is 0 and count them
    not_anticipated_counts[f'Q{i}_solution'] = df1[df1[anticipate_col_name] == 0].shape[0]

print(not_anticipated_counts)


{'Q1_solution': 769, 'Q2_solution': 958, 'Q3_solution': 567, 'Q4_solution': 929, 'Q5_solution': 1030, 'Q6_solution': 1192}


## Rule to split vocabulary

In [ ]:
def split_expression(expression):
  if pd.isnull(expression) or expression is None:
    return []
  return [char for char in expression if not char.isspace()]
    # Regular expression pattern to split the expression into numbers, symbols, and whitespaces
    # pattern = r'(\d+\/\d+|\d+|\s|[^\d\s]+)'

    # Find all matches using the pattern
    # parts = re.findall(pattern, expression)

    #return parts

expression = "1/21*(7*x+9)^3 + c"
print(split_expression(expression))

['1', '/', '2', '1', '*', '(', '7', '*', 'x', '+', '9', ')', '^', '3', '+', 'c']


## Filter dataset: wrong answer only

## Create contingency table

In [ ]:
def make_contingency_table(df, column_name):
    # Get unique characters from all cells in the column
    all_chars = "".join(df[column_name].apply(split_expression).sum())
    unique_chars = list(set(all_chars))

    # Use a dictionary to collect data for the dataframe
    data_dict = {}

    # For each test, count occurrences of each character in the solution
    for test, solution in df[column_name].items():
        chars = split_expression(solution)
        data_dict[test] = [chars.count(char) for char in unique_chars]

    # Convert the dictionary to a DataFrame
    contingency = pd.DataFrame(data_dict, index=unique_chars)

    return contingency.sort_index()


contingency_tables = {}
for i in range(1, 7):  # Adjust range based on the number of Q's.
    filtered_df = df1[df1[f'Q{i}_anticipate'].isin([0, 1])]
    contingency_tables[f'Q{i}_solution'] = make_contingency_table(filtered_df, f'Q{i}_solution')

# Print or use the tables
# for key, table in contingency_tables.items():
#     print(f"Contingency table for {key}:")
#     print(table)
#     print("\n")
print(contingency_tables['Q1_solution'])

   T2  T9  T10  T12  T15  T16  T18  T19  T21  T22  ...  T2454  T2455  T2456  \
(   1   0    1    1    1    1    1    0    1    2  ...      2      1      3   
)   1   0    1    1    1    1    1    0    1    2  ...      2      1      3   
*   1   3    3    4    6    1    2    3    1    6  ...      6      4      6   
+   2   3    3    5    6    2    3    3    2    6  ...      6      4      6   
/   1   0    1    1    1    1    1    0    1    2  ...      2      1      3   
0   0   0    3    1    3    0    3    0    0    3  ...      3      0      2   
1   0   0    1    3    1    0    2    0    1    1  ...      3      1      2   
2   2   3    2    3    4    1    1    1    1    4  ...      6      3      7   
3   0   1    2    1    2    0    2    2    1    6  ...      7      4      6   
4   1   0    1    2    4    1    0    1    0    1  ...      2      4      3   
5   0   0    0    3    1    0    0    0    0    1  ...      1      0      3   
6   0   0    0    2    4    1    0    1    0    2  .

In [ ]:
print(contingency_tables['Q1_solution'])

   T0  T2  T3  T4  T12  T13  T14  T17  T20  T23  ...  T2440  T2441  T2442  \
(   2   2   2   2    2    2    2    2    2    2  ...      2      2      2   
)   2   2   2   2    2    2    2    2    2    2  ...      2      2      2   
*   1   1   1   1    1    1    1    1    1    1  ...      1      1      1   
+   1   1   1   1    1    1    1    1    1    1  ...      1      1      1   
0   0   0   0   0    0    0    0    0    0    0  ...      0      0      0   
1   0   0   1   1    0    2    1    1    0    0  ...      1      1      0   
2   0   0   0   0    0    0    0    0    0    0  ...      0      0      1   
3   0   0   1   0    0    0    0    0    0    0  ...      1      1      0   
4   1   0   0   0    0    0    0    0    0    0  ...      0      0      0   
5   0   0   0   1    0    0    1    1    0    0  ...      0      0      0   
6   0   0   0   0    0    0    0    0    1    0  ...      0      0      0   
7   0   0   0   0    0    0    0    0    0    0  ...      0      0      0   

# Correspondence Analysis

The goal of CA is to **represent the categories of characters as points of a lower-dimension Euclidean space** (usually 2D) in such a way that the Euclidean distance between two points (approximately) corresponds to the $\chi^2$
distance between the corresponding categories.

Correspondence analysis is a popular, well-known, and classical tool in textual data analysis.
* CA can be exploited to achieve a content mapping as it is useful to represent the system of relationships among texts, among words and between texts and words.
* CA, operating on rows (words) and columns (texts or subcorpora) of a word text matrix (TDM, ×
contingency table), transforms the frequencies (cells of the matrix) in coordinates on a multidimensional system of Cartesian axes.
* CA displays the texts/words in a low-dimensional space by mapping an appropriate distance - χ
(chi-square) distance (calculated between pairs of rows and pairs of columns) into a (weighted) Euclidean distance, whence it projects texts and/or words on a Cartesian plane.
* CA aims at detecting latent dimensions underlying the data which summarize the association pattern between variables (dimensional reduction).
* CA is an analog of principal component analysis with a contingency table as imput.
* CA is based on a singular value decomposition or, equivalently, on a multidimensional scaling on the chi square-based distance matrices.


### **Reading the CA graph**
We briefy summarize some points for reading the graphs obtained from CA:
1. the position of a word or text has a role only within the global context of the graph (that is, **it makes no sense by itself, but it does make sense in comparison with the positions taken by all the other points wrt the centroid** positioned at the origin of the axes);
2. the words or responses that have contributed most to the solution and that, therefore, can be considered more important in the context reconstructed by the graph, are those far from the origin of the axes ;
3. if **two words are close they have similar profiles** ;
4. if **two responses are close they have similar (lexical) profiles** ;
5. The mutual position taken by a word and a responses cannot be evaluated directly because the distance between the two points on the graph does not translate automatically in terms of similarity: it must be evaluated with reference to the positions taken by all the other elements.

Check link: https://cran.r-project.org/web/packages/visae/vignettes/ca_biplots.html

In [ ]:
pip install prince

## CA: Each student responses VS chars

In [ ]:
import prince

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Perform Correspondence Analysis on each contingency table
ca_objects = {}
for col, table in contingency_tables.items():
    print(f"Performing CA for {col}...")
    ca = prince.CA(n_components=2)
    ca = ca.fit(table)
    ca_objects[col] = ca

Performing CA for Q1_solution...
Performing CA for Q2_solution...
Performing CA for Q3_solution...
Performing CA for Q4_solution...
Performing CA for Q5_solution...
Performing CA for Q6_solution...


### Dynamic uniplots

In [ ]:
pip install plotly

In [ ]:
import plotly.graph_objects as go

for col, ca in ca_objects.items():
    print(f"Plotting for {col}...")

    # Get the coordinates for columns (TestNo)
    col_coordinates = ca.column_coordinates(contingency_tables[col])

    # Extract Qi_anticipate values for each TestNo
    testnos_in_col_coordinates = col_coordinates.index.tolist()
    filtered_df = df1.loc[testnos_in_col_coordinates]

    anticipate_col_name = col.replace('solution', 'anticipate')
    anticipate_values = filtered_df[anticipate_col_name].tolist()

    # Split coordinates based on anticipate values
    anticipated_coords = col_coordinates[filtered_df[anticipate_col_name] == 1]
    not_anticipated_coords = col_coordinates[filtered_df[anticipate_col_name] == 0]

    # Create interactive plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=anticipated_coords.iloc[:, 0],
                             y=anticipated_coords.iloc[:, 1],
                             mode='markers',
                             marker=dict(color='purple', size=10),
                             name='Anticipate = 1',
                             text=anticipated_coords.index,  # TestNo as hover text
                             hoverinfo='text'))

    fig.add_trace(go.Scatter(x=not_anticipated_coords.iloc[:, 0],
                             y=not_anticipated_coords.iloc[:, 1],
                             mode='markers',
                             marker=dict(color='green', size=10),
                             name='Anticipate = 0',
                             text=not_anticipated_coords.index,  # TestNo as hover text
                             hoverinfo='text'))

    # Add titles and labels
    fig.update_layout(title=f'Correspondence Analysis for {col}',
                      xaxis_title='Dim 1',
                      yaxis_title='Dim 2',
                      legend_title="Anticipate Values",
                      showlegend=True)

    # Display interactive plot
    fig.show()

Plotting for Q1_solution...


Plotting for Q2_solution...


Plotting for Q3_solution...


Plotting for Q4_solution...


Plotting for Q5_solution...


Plotting for Q6_solution...


# K-means for clustering

Effective types of anticipated responses:

Q1: 1, Q2: 3, Q3: 4, Q4: 4, Q5: 2, Q6: 3.

In [ ]:
from sklearn.cluster import KMeans

# Define the number of clusters for each "Response" column
k_values = {
    'Q1_solution': 2,# vs 1 type of anticipated response
    'Q2_solution': 3,# vs 3 ""
    'Q3_solution': 4,# vs 4 ""
    'Q4_solution': 3,# vs 4
    'Q5_solution': 2,# vs 2
    'Q6_solution': 2 # vs 3

}

### Dynamic plots

In [ ]:
import plotly.graph_objs as go

clustered_columns = {}

for col, ca in ca_objects.items():
    columns_coord = ca.column_coordinates(contingency_tables[col])

    k = k_values[col]
    kmeans = KMeans(n_clusters=k, n_init=10)
    clusters = kmeans.fit_predict(columns_coord)
    clustered_columns[col] = clusters

    traces = []  # List to store traces for each cluster

    # Create hover text using 'TestNo'
    hover_text = columns_coord.index

    for cluster_number in range(k):
        cluster_indices = (clusters == cluster_number)
        cluster_name = chr(65 + cluster_number)  # Convert cluster number to alphabetical name

        trace = go.Scatter(
            x=columns_coord[0][cluster_indices],
            y=columns_coord[1][cluster_indices],
            mode='markers',
            text=hover_text[cluster_indices],  # This makes the labels appear upon hovering
            marker=dict(
                size=10,
                color=cluster_number,  # Color based on cluster number
                colorscale='Rainbow',
                showscale=False  # Hide individual colorbar
            ),
            name=cluster_name  # Name of the trace in the legend
        )

        traces.append(trace)

    layout = go.Layout(
        title=f"Column Clusters for {col}",
        xaxis=dict(title='Component 1'),
        yaxis=dict(title='Component 2')
    )

    fig = go.Figure(data=traces, layout=layout)
    fig.show()



## Add columns and statistics


In [ ]:
dfs = {}

for col, ca in ca_objects.items():
    columns_coord = ca.column_coordinates(contingency_tables[col])

    k = k_values[col]
    kmeans = KMeans(n_clusters=k, n_init=10)
    clusters = kmeans.fit_predict(columns_coord)

    # Assuming clusters now has the same length as columns_coord
    df_cluster = pd.DataFrame(data=clusters, index=columns_coord.index, columns=['Cluster'])

    # Rename clusters alphabetically
    unique_clusters = df_cluster['Cluster'].unique()
    cluster_mapping = {cluster: chr(65 + idx) for idx, cluster in enumerate(unique_clusters)}
    df_cluster['Cluster'] = df_cluster['Cluster'].map(cluster_mapping)

    anticipate_col_name = col.replace('solution', 'anticipate')
    df_cluster[anticipate_col_name] = df1.loc[df_cluster.index, anticipate_col_name]

    dfs[col] = df_cluster

    print(f"For {col}:")
    print(df_cluster['Cluster'].value_counts())
    print("\n")



For Q1_solution:
A    568
B    233
Name: Cluster, dtype: int64


For Q2_solution:
B    597
A    226
C    139
Name: Cluster, dtype: int64


For Q3_solution:
A    418
D    174
C    114
B     82
Name: Cluster, dtype: int64


For Q4_solution:
C    411
B    368
A    227
Name: Cluster, dtype: int64


For Q5_solution:
A    622
B    409
Name: Cluster, dtype: int64


For Q6_solution:
A    1121
B     124
Name: Cluster, dtype: int64




In [ ]:
for col, df in dfs.items():
    print(f"Dataframe for {col}:")
    print(df.head())
    print("\n" + "-"*50 + "\n")


Dataframe for Q1_solution:
    Cluster  Q1_anticipate
T0        A              0
T2        B              0
T3        A              0
T4        A              0
T12       B              0

--------------------------------------------------

Dataframe for Q2_solution:
    Cluster  Q2_anticipate
T2        A              0
T9        B              0
T10       C              0
T12       B              0
T15       B              0

--------------------------------------------------

Dataframe for Q3_solution:
    Cluster  Q3_anticipate
T8        A              0
T10       B              0
T12       A              0
T14       A              0
T19       A              1

--------------------------------------------------

Dataframe for Q4_solution:
   Cluster  Q4_anticipate
T1       A              0
T2       B              0
T3       A              0
T4       A              0
T7       C              0

--------------------------------------------------

Dataframe for Q5_solution:
    Cluster

In [ ]:
for col, df in dfs.items():
    filename = f"{col}_clustered.csv"
    df.to_csv(filename)
    print(f"Dataframe for {col} saved to {filename}")


Dataframe for Q1_solution saved to Q1_solution_clustered.csv
Dataframe for Q2_solution saved to Q2_solution_clustered.csv
Dataframe for Q3_solution saved to Q3_solution_clustered.csv
Dataframe for Q4_solution saved to Q4_solution_clustered.csv
Dataframe for Q5_solution saved to Q5_solution_clustered.csv
Dataframe for Q6_solution saved to Q6_solution_clustered.csv


In [ ]:
filename = "Q1_solution_clustered.csv"
df_q1_solution = pd.read_csv(filename, index_col=0)  # Assuming the first column in the CSV is the index (TestNo in this case)

# Display the first few rows to check
print(df_q1_solution.head())

    Cluster  Q1_anticipate
T0        A              0
T2        B              0
T3        A              0
T4        A              0
T12       B              0


In [ ]:
# Initialize the original df1 with added columns
for col, df in dfs.items():
    # For each unique cluster in the dataframe, create a new column in df1
    for cluster in df['Cluster'].unique():
        # Construct the new column name
        column_name = f"{col.split('_')[0]}_{cluster}"

        # Initialize the column with 0s
        df1[column_name] = 0

        # For rows where the TestNo is in the cluster, set the value to 1
        testnos_in_cluster = df[df['Cluster'] == cluster].index
        df1.loc[df1.index.isin(testnos_in_cluster), column_name] = 1

# To check the changes
print(df1.head())

       Username      Started.on       Completed        Time.taken  Grade.6.00  \
TestNo                                                                          
T0      user004  10-10-22 10:10  10-10-22 10:57   46 mins 30 secs         5.0   
T1      user004  10-10-22 14:16  10-10-22 14:35   18 mins 34 secs         3.0   
T2      user006  10-08-22 14:07   10-09-22 0:53  10 hours 45 mins         1.0   
T3      user006   10-09-22 8:37   10-09-22 9:10   32 mins 59 secs         3.0   
T4      user006   10-09-22 9:15   10-09-22 9:43   28 mins 18 secs         5.0   

        Q1_seed                   Q1_problem       Q1_solution  \
TestNo                                                           
T0       808626   \(\int{\frac{4}{p}} d{a}\)   4*log(abs(p))+c   
T1       490862  \(\int{\frac{10}{y}} d{a}\)  10*log(abs(y))+c   
T2       397885   \(\int{\frac{9}{r}} d{a}\)   9*log(abs(r))+c   
T3       154367  \(\int{\frac{13}{t}} d{a}\)  13*log(abs(t))+c   
T4       342032  \(\int{\frac{15}{t}

In [ ]:
df1.head()

,Username,Started.on,Completed,Time.taken,Grade.6.00,Q1_seed,Q1_problem,Q1_solution,Q1_mistakes,Q1_ans,...,Q3_B,Q3_C,Q3_D,Q4_A,Q4_B,Q4_C,Q5_A,Q5_B,Q6_A,Q6_B
TestNo,,,,,,,,,,,,,,,,,,,,,
T0,user004,10-10-22 10:10,10-10-22 10:57,46 mins 30 secs,5.0,808626,\(\int{\frac{4}{p}} d{a}\),4*log(abs(p))+c,\({4\cdot \ln \left( \left| p\right| \right)}\),0 + c,...,0,0,0,0,0,0,0,0,0,0
T1,user004,10-10-22 14:16,10-10-22 14:35,18 mins 34 secs,3.0,490862,\(\int{\frac{10}{y}} d{a}\),10*log(abs(y))+c,\({10\cdot \ln \left( \left| y\right| \right)}\),10*log*(/y/)+c,...,0,0,0,1,0,0,1,0,0,0
T2,user006,10-08-22 14:07,10-09-22 0:53,10 hours 45 mins,1.0,397885,\(\int{\frac{9}{r}} d{a}\),9*log(abs(r))+c,\({9\cdot \ln \left( \left| r\right| \right)}\),0,...,0,0,0,0,1,0,1,0,1,0
T3,user006,10-09-22 8:37,10-09-22 9:10,32 mins 59 secs,3.0,154367,\(\int{\frac{13}{t}} d{a}\),13*log(abs(t))+c,\({13\cdot \ln \left( \left| t\right| \right)}\),13*ln(t)+c,...,0,0,0,1,0,0,0,0,1,0
T4,user006,10-09-22 9:15,10-09-22 9:43,28 mins 18 secs,5.0,342032,\(\int{\frac{15}{t}} d{a}\),15*log(abs(t))+c,\({15\cdot \ln \left( \left| t\right| \right)}\),15*ln(|t|)+c,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
df1.to_csv('cluster_data.csv', index=True)